# Run Any Kind of OLS Regression (ANOVA, GLM, etc.)

### Authors: Calvin Howard.

#### Last updated: February 1, 2025

Use this to run/test a statistical model (e.g., regression or T-tests) on a spreadsheet.

Notes:
- To best use this notebook, you should be familar with GLM design and Contrast Matrix design. See this webpage to get started:
[FSL's GLM page](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM)

Prepare output directory

In [1]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook05'

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Import Data

In [2]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/revisionsdata.csv'
sheet = None

In [3]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_data()
data_df

,Dataset,Subject,Nifti_File_Path,age,z_scored_improvement,sbc_conn,sex,DatasetInt,Baseline_Cognitive_Score,Frequency,Pulse_Width__uS_,Amperage__mA_
0,AD Fornix DBS,1203,/Users/cu135/Partners HealthCare Dropbox/Calvi...,76,-72.607386,70.124471,m,1,28.0,130,90,3.5
1,AD Fornix DBS,1202,/Users/cu135/Partners HealthCare Dropbox/Calvi...,74,-1.529691,51.103184,m,1,22.0,130,90,3.5
2,AD Fornix DBS,1201,/Users/cu135/Partners HealthCare Dropbox/Calvi...,78,-6.416910,52.243723,m,1,19.0,130,90,3.5
3,AD Fornix DBS,150,/Users/cu135/Partners HealthCare Dropbox/Calvi...,71,14.362311,73.488381,m,1,17.0,130,90,3.5
4,AD Fornix DBS,149,/Users/cu135/Partners HealthCare Dropbox/Calvi...,77,-89.274052,62.007555,m,1,19.0,130,90,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...
78,PD STN DBS,MDST05,NaN,60,0.503206,NaN,m,2,143.0,150,50,3.5
79,PD STN DBS,MDST04,/Users/cu135/Partners HealthCare Dropbox/Calvi...,50,-0.282257,21.207602,m,2,NaN,130,60,3.5
80,PD STN DBS,MDST03,/Users/cu135/Partners HealthCare Dropbox/Calvi...,62,-0.005890,30.900051,f,2,0.0,130,60,3.5
81,PD STN DBS,MDST02,/Users/cu135/Partners HealthCare Dropbox/Calvi...,50,1.294321,16.295870,f,2,2.0,130,60,3.5


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [4]:
data_df.columns

Index(['Dataset', 'Subject', 'Nifti_File_Path', 'age', 'z_scored_improvement',
       'sbc_conn', 'sex', 'DatasetInt', 'Baseline_Cognitive_Score',
       'Frequency', 'Pulse_Width__uS_', 'Amperage__mA_'],
      dtype='object')

In [5]:
drop_list = ['Nifti_File_Path', 'z_scored_improvement', 'age', 'Dataset']

In [6]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

,Dataset,Subject,Nifti_File_Path,age,z_scored_improvement,sbc_conn,sex,DatasetInt,Baseline_Cognitive_Score,Frequency,Pulse_Width__uS_,Amperage__mA_
0,AD Fornix DBS,1203,/Users/cu135/Partners HealthCare Dropbox/Calvi...,76,-72.607386,70.124471,m,1,28.0,130,90,3.5
1,AD Fornix DBS,1202,/Users/cu135/Partners HealthCare Dropbox/Calvi...,74,-1.529691,51.103184,m,1,22.0,130,90,3.5
2,AD Fornix DBS,1201,/Users/cu135/Partners HealthCare Dropbox/Calvi...,78,-6.416910,52.243723,m,1,19.0,130,90,3.5
3,AD Fornix DBS,150,/Users/cu135/Partners HealthCare Dropbox/Calvi...,71,14.362311,73.488381,m,1,17.0,130,90,3.5
4,AD Fornix DBS,149,/Users/cu135/Partners HealthCare Dropbox/Calvi...,77,-89.274052,62.007555,m,1,19.0,130,90,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...
77,PD STN DBS,MDST06,/Users/cu135/Partners HealthCare Dropbox/Calvi...,60,0.245073,23.577739,m,2,143.0,150,50,3.5
79,PD STN DBS,MDST04,/Users/cu135/Partners HealthCare Dropbox/Calvi...,50,-0.282257,21.207602,m,2,NaN,130,60,3.5
80,PD STN DBS,MDST03,/Users/cu135/Partners HealthCare Dropbox/Calvi...,62,-0.005890,30.900051,f,2,0.0,130,60,3.5
81,PD STN DBS,MDST02,/Users/cu135/Partners HealthCare Dropbox/Calvi...,50,1.294321,16.295870,f,2,2.0,130,60,3.5


**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [7]:
data_df.columns

Index(['Dataset', 'Subject', 'Nifti_File_Path', 'age', 'z_scored_improvement',
       'sbc_conn', 'sex', 'DatasetInt', 'Baseline_Cognitive_Score',
       'Frequency', 'Pulse_Width__uS_', 'Amperage__mA_'],
      dtype='object')

Set the parameters for dropping rows

In [8]:
column = 'Dataset'  # The column you'd like to evaluate
condition = 'equal'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'PD STN DBS' # The value to drop if found

In [9]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

,Dataset,Subject,Nifti_File_Path,age,z_scored_improvement,sbc_conn,sex,DatasetInt,Baseline_Cognitive_Score,Frequency,Pulse_Width__uS_,Amperage__mA_
0,AD Fornix DBS,1203,/Users/cu135/Partners HealthCare Dropbox/Calvi...,76,-72.607386,70.124471,m,1,28.0,130,90,3.5
1,AD Fornix DBS,1202,/Users/cu135/Partners HealthCare Dropbox/Calvi...,74,-1.529691,51.103184,m,1,22.0,130,90,3.5
2,AD Fornix DBS,1201,/Users/cu135/Partners HealthCare Dropbox/Calvi...,78,-6.416910,52.243723,m,1,19.0,130,90,3.5
3,AD Fornix DBS,150,/Users/cu135/Partners HealthCare Dropbox/Calvi...,71,14.362311,73.488381,m,1,17.0,130,90,3.5
4,AD Fornix DBS,149,/Users/cu135/Partners HealthCare Dropbox/Calvi...,77,-89.274052,62.007555,m,1,19.0,130,90,3.5
5,AD Fornix DBS,148,/Users/cu135/Partners HealthCare Dropbox/Calvi...,51,-206.966360,75.739873,m,1,13.0,130,90,3.5
6,AD Fornix DBS,147,/Users/cu135/Partners HealthCare Dropbox/Calvi...,59,-4.035957,69.447270,m,1,13.0,130,90,3.5
7,AD Fornix DBS,146,/Users/cu135/Partners HealthCare Dropbox/Calvi...,76,-53.819507,46.331586,m,1,24.0,130,90,3.5
8,AD Fornix DBS,145,/Users/cu135/Partners HealthCare Dropbox/Calvi...,74,6.281503,85.082502,m,1,23.0,130,90,3.5
9,AD Fornix DBS,144,/Users/cu135/Partners HealthCare Dropbox/Calvi...,79,-60.812514,58.899731,m,1,13.0,130,90,3.5


**Standardize Data**
- Enter Columns you Don't want to standardize into a list
- group_col is the column containing a category for each dataset. It ensures standardization is performed within each group.

In [10]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['Nifti_File_Path', 'Subject']
group_col = 'Dataset'

In [11]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize, group_col=group_col)
data_df

Unable to standardize column sex.


/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/calvin_utils_project/calvin_utils_project/calvin_utils/permutation_analysis_utils/statsmodels_palm.py:182: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 1.06577488  0.81530501  1.31624476  0.43960019  1.19100982 -2.06509857
 -1.06321907  1.06577488  0.81530501  1.4414797   0.43960019  1.19100982
  0.56483513  0.69007007 -1.188454    0.56483513 -1.31368894 -1.31368894
 -0.1865745   0.81530501  0.06389538 -0.1865745   0.18913031  0.56483513
  0.81530501  0.18913031  0.69007007 -0.81274919 -2.56603832 -1.188454
  0.56483513  0.06389538  0.94053994 -1.93986363 -0.93798413 -0.06133956
  0.18913031  0.18913031 -0.68751425  0.56483513  0.56483513 -0.93798413
 -0.43704438 -0.1865745  -2.19033351 -0.31180944  1.06577488  1.19100982
 -0.68751425]' has dtype incompatible with int64, please explicitly cast to a compatible dtype fi

,Dataset,Subject,Nifti_File_Path,age,z_scored_improvement,sbc_conn,sex,DatasetInt,Baseline_Cognitive_Score,Frequency,Pulse_Width__uS_,Amperage__mA_
0,AD Fornix DBS,1203,/Users/cu135/Partners HealthCare Dropbox/Calvi...,1.065775,-0.773684,0.362496,m,NaN,-0.032593,-0.298142,0.298142,NaN
1,AD Fornix DBS,1202,/Users/cu135/Partners HealthCare Dropbox/Calvi...,0.815305,0.681280,-1.872812,m,NaN,-0.210047,-0.298142,0.298142,NaN
2,AD Fornix DBS,1201,/Users/cu135/Partners HealthCare Dropbox/Calvi...,1.316245,0.581238,-1.738780,m,NaN,-0.298773,-0.298142,0.298142,NaN
3,AD Fornix DBS,150,/Users/cu135/Partners HealthCare Dropbox/Calvi...,0.439600,1.006590,0.757809,m,NaN,-0.357924,-0.298142,0.298142,NaN
4,AD Fornix DBS,149,/Users/cu135/Partners HealthCare Dropbox/Calvi...,1.191010,-1.114852,-0.591373,m,NaN,-0.298773,-0.298142,0.298142,NaN
5,AD Fornix DBS,148,/Users/cu135/Partners HealthCare Dropbox/Calvi...,-2.065099,-3.524021,1.022396,m,NaN,-0.476226,-0.298142,0.298142,NaN
6,AD Fornix DBS,147,/Users/cu135/Partners HealthCare Dropbox/Calvi...,-1.063219,0.629977,0.282914,m,NaN,-0.476226,-0.298142,0.298142,NaN
7,AD Fornix DBS,146,/Users/cu135/Partners HealthCare Dropbox/Calvi...,1.065775,-0.389095,-2.433551,m,NaN,-0.150895,-0.298142,0.298142,NaN
8,AD Fornix DBS,145,/Users/cu135/Partners HealthCare Dropbox/Calvi...,0.815305,0.841176,2.120305,m,NaN,-0.180471,-0.298142,0.298142,NaN
9,AD Fornix DBS,144,/Users/cu135/Partners HealthCare Dropbox/Calvi...,1.441480,-0.532243,-0.956592,m,NaN,-0.476226,-0.298142,0.298142,NaN


# 02 - Define Your Formula

**Critical: the dependent (y) variable should always be the column with the neuroimaging files in it**

This is the formula relating outcome to predictors, and takes the form:
- y = B0 + B1 + B2 + B3 + . . . BN

It is defined using the columns of your dataframe instead of the variables above:
- 'Apples_Picked ~ hours_worked + owns_apple_picking_machine'

____
**ANOVA**
- Tests differences in means for one categorical variable.
- formula = 'Outcome ~ C(Group1)'

**2-Way ANOVA**
- Tests differences in means for two categorical variables without interaction.
- formula = 'Outcome ~ C(Group1) + C(Group2)'

**2-Way ANOVA with Interaction**
- Tests for interaction effects between two categorical variables.
- formula = 'Outcome ~ C(Group1) * C(Group2)'

**ANCOVA**
- Similar to ANOVA, but includes a covariate to control for its effect.
- formula = 'Outcome ~ C(Group1) + Covariate'

**2-Way ANCOVA**
- Extends ANCOVA with two categorical variables and their interaction, controlling for a covariate.
- formula = 'Outcome ~ C(Group1) * C(Group2) + Covariate'

**Multiple Regression**
- Assesses the impact of multiple predictors on an outcome.
- formula = 'Outcome ~ Predictor1 + Predictor2'

**Simple Linear Regression**
- Assesses the impact of a single predictor on an outcome.
- formula = 'Outcome ~ Predictor'

**MANOVA**
- Assesses multiple dependent variables across groups.
- Note: Not typically set up with a formula in statsmodels. Requires specialized functions.

____
Use the printout below to design your formula. 
- Left of the "~" symbol is the thing to be predicted. 
- Right of the "~" symbol are the predictors. 
- ":" indicates an interaction between two things. 
- "*" indicates and interactions AND it accounts for the simple effects too. 
- "+" indicates that you want to add another predictor. 

In [12]:
data_df.columns

Index(['Dataset', 'Subject', 'Nifti_File_Path', 'age', 'z_scored_improvement',
       'sbc_conn', 'sex', 'DatasetInt', 'Baseline_Cognitive_Score',
       'Frequency', 'Pulse_Width__uS_', 'Amperage__mA_'],
      dtype='object')

** the left side of the equation is expected to be called 'Nifti_File_Path'. This should be in your CSV as a column, spelled the same way. **

In [13]:
formula = "Nifti_File_Path ~ age*z_scored_improvement + Dataset"

# 02 - Visualize Your Design Matrix

This is the explanatory variable half of your regression formula
_______________________________________________________
Create Design Matrix: Use the create_design_matrix method. You can provide a list of formula variables which correspond to column names in your dataframe.

- design_matrix = palm.create_design_matrix(formula_vars=["var1", "var2", "var1*var2"])
- To include interaction terms, use * between variables, like "var1*var2".
- By default, an intercept will be added unless you set intercept=False
- **don't explicitly add the 'intercept' column. I'll do it for you.**
- If you want to compare specific datasets within a column, leave 'coerce_str'=False

In [14]:
# Define the design matrix
outcome_matrix, design_matrix = cal_palm.define_design_matrix(formula, data_df=data_df, voxelwise_variable_list=['Nifti_File_Path'], coerce_str=False)
design_matrix

,Intercept,age,z_scored_improvement,age:z_scored_improvement
0,1.0,1.065775,-0.773684,-0.824573
1,1.0,0.815305,0.681280,0.555451
2,1.0,1.316245,0.581238,0.765052
3,1.0,0.439600,1.006590,0.442497
4,1.0,1.191010,-1.114852,-1.327800
5,1.0,-2.065099,-3.524021,7.277450
6,1.0,-1.063219,0.629977,-0.669803
7,1.0,1.065775,-0.389095,-0.414688
8,1.0,0.815305,0.841176,0.685815
9,1.0,1.441480,-0.532243,-0.767217


# 03 - Visualize Your Dependent Variable

I have generated this for you based on the formula you provided

In [15]:
outcome_matrix

,Nifti_File_Path
0,/Users/cu135/Partners HealthCare Dropbox/Calvi...
1,/Users/cu135/Partners HealthCare Dropbox/Calvi...
2,/Users/cu135/Partners HealthCare Dropbox/Calvi...
3,/Users/cu135/Partners HealthCare Dropbox/Calvi...
4,/Users/cu135/Partners HealthCare Dropbox/Calvi...
5,/Users/cu135/Partners HealthCare Dropbox/Calvi...
6,/Users/cu135/Partners HealthCare Dropbox/Calvi...
7,/Users/cu135/Partners HealthCare Dropbox/Calvi...
8,/Users/cu135/Partners HealthCare Dropbox/Calvi...
9,/Users/cu135/Partners HealthCare Dropbox/Calvi...


# 04 - Generate Contrasts

Generate a Contrast Matrix
- This is different from the contrast matrices used in cell-means regressions such as in PALM, but it is much more powerful. 



For more information on contrast matrices, please refer to this: https://cran.r-project.org/web/packages/codingMatrices/vignettes/codingMatrices.pdf

Generally, these drastically effect the results of ANOVA. However, they are mereley a nuisance for a regression.
In essence, they assess if coefficients are significantly different

________________________________________________________________
A coding matrix (a contrast matrix if it sums to zero) is simply a way of defining what coefficients to evaluate and how to evaluate them. 
If a coefficient is set to 1 and everything else is set to zero, we are taking the mean of the coefficient's means and assessing if they significantly
deviate from zero--IE we are checking if it had a significant impact on the ability to predict the depdendent variable.
If a coefficient is set to 1, another is -1, and others are 0, we are assessing how the means of the two coefficients deviate from eachother. 
If several coefficients are 1 and several others are -1, we are assessing how the group-level means of the two coefficients deviate from eachother.
If a group of coefficients are 1, a group is -1, and a group is 0, we are only assessing how the groups +1 and -1 have differing means. 

1: This value indicates that the corresponding variable's coefficient in the model is included in the contrast. It means you are interested in estimating the effect of that variable.

0: This value indicates that the corresponding variable's coefficient in the model is not included in the contrast. It means you are not interested in estimating the effect of that variable.

-1: This value indicates that the corresponding variable's coefficient in the model is included in the contrast, but with an opposite sign. It means you are interested in estimating the negative effect of that variable.

----------------------------------------------------------------
The contrast matrix is typically a matrix with dimensions (number of contrasts) x (number of regression coefficients). Each row of the contrast matrix represents a contrast or comparison you want to test.

For example, let's say you have the following regression coefficients in your model:

Intercept, Age, connectivity, Age_interaction_connectivity
A contrast matric has dimensions of [n_predictors, n_experiments] where each experiment is a contrast

If you want to test the hypothesis that the effect of Age is significant, you can set up a contrast matrix with a row that specifies this contrast (actually an averaging vector):
```
[0,1,0,0]. This is an averaging vector because it sums to 1
```
This contrast will test the coefficient corresponding to the Age variable against zero.


If you want to test the hypothesis that the effect of Age is different from the effect of connectivity, you can set up a contrast matrix with two rows:
```
[0,1,−1,0]. This is a contrast because it sums to 0
```

Thus, if you want to see if any given effect is significant compared to the intercept (average), you can use the following contrast matrix:
```
[1,0,0,0]
[-1,1,0,0]
[-1,0,1,0]
[-1,0,0,1] actually a coding matrix of averaging vectors
```

The first row tests the coefficient for Age against zero, and the second row tests the coefficient for connectivity against zero. The difference between the two coefficients can then be assessed.
_____
You can define any number of contrasts in the contrast matrix to test different hypotheses or comparisons of interest in your regression analysis.

It's important to note that the specific contrasts you choose depend on your research questions and hypotheses. You should carefully consider the comparisons you want to make and design the contrast matrix accordingly.

- Examples:
    - [Two Sample T-Test](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM#Two-Group_Difference_.28Two-Sample_Unpaired_T-Test.29)
    - [One Sample with Covariate](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM#Single-Group_Average_with_Additional_Covariate)

In [16]:
contrast_matrix = cal_palm.generate_basic_contrast_matrix(design_matrix)

Here is a basic contrast matrix set up to evaluate the significance of each variable.
Here is an example of what your contrast matrix looks like as a dataframe: 


,Intercept,age,z_scored_improvement,age:z_scored_improvement
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


Below is the same contrast matrix, but as an array.
Copy it into a cell below and edit it for more control over your analysis.
[
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
]


Edit Contrast Matrix Here
- The generic contrast matrix will simply check if your Betas are significantly different from the intercept (average)

In [17]:
# contrast_matrix = [
#     [1, 0, 0, 0, 0],
#     [0, 1, 0, 0, 0],
#     [0, 0, 1, 0, 0],
#     [0, 0, 0, 1, 0],
#     [0, 0, 0, 0, 1],
# ]

Finalize Contrast Matrix

In [18]:
contrast_matrix_df = cal_palm.finalize_contrast_matrix(design_matrix=design_matrix, 
                                                    contrast_matrix=contrast_matrix) 
contrast_matrix_df

,Intercept,age,z_scored_improvement,age:z_scored_improvement
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


# 06 - Save the Files

Standardization during regression is critical. 
- data_transform_method='standardize' will ensure the voxelwise values are standardized
    - if you design matrix has a column called 'Dataset', the standardization will standardize values within each dataset individually, which is as should be done normally.
    - If you call data_transform_method='standardize' without having a 'Dataset' column in your design matrix, the entire collection of images will be standardized. This is potentially dangerous and misleading. Be careful, and consider not standardizing at all, or going back and adding a 'Dataset' column. 

Mask Path
- set mask_path to the path of your local brain mask which matches the resolution of the files you have collected. Typically this is an MNI 152 brain mask. 
    - download one here: https://nilearn.github.io/dev/modules/generated/nilearn.datasets.load_mni152_brain_mask.html

In [19]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'
data_transform_method='standardize'

In [20]:
from calvin_utils.ccm_utils.npy_utils import RegressionNPYPreparer
preparer = RegressionNPYPreparer(
    design_matrix=design_matrix,
    contrast_matrix=contrast_matrix_df,
    outcome_matrix=outcome_matrix,
    out_dir=out_dir,
    mask_path=mask_path,
    exchangeability_blocks=None,   # or your DataFrame
    data_transform_method=data_transform_method
)
dataset_dict, json_path = preparer.run()

Design matrix saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook05/main/design_matrix.npy
Contrast matrix saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook05/main/contrast_matrix.npy


Loading NIFTI files: 100%|██████████| 49/49 [00:00<00:00, 100.26it/s]


Stacked NIFTI data saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook05/main/niftis.npy
Dataset dictionary saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook05/dataset_dict.json


# 07 - Run the Regression

Mask Path
- set mask_path to the path of your local brain mask which matches the resolution of the files you have collected. Typically this is an MNI 152 brain mask. 
    - download one here: https://nilearn.github.io/dev/modules/generated/nilearn.datasets.load_mni152_brain_mask.html

In [21]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'


In [23]:
from calvin_utils.ccm_utils.npy_regression import RegressionNPYAnalysis
reg = RegressionNPYAnalysis(data_dict_path=json_path,
                      n_permutations=2, 
                      out_dir=out_dir,
                      fwe=True,
                      max_stat_method="pseudo_var_smooth",
                      mask_path=mask_path,
                      verbose=False)
results = reg.run()
print("Voxelwise FWE p-values shape:", results["voxelwise_p_values"].shape)

There are 49 subjects
There are 4 covariates
There are 228483 voxels
X (observations, covariates): (49, 4)
Y (voxels, observations): (228483, 49)

 Contrast matrix: 
 f[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]] 

Using Family-wise error correction: True with method: pseudo_var_smooth

 Saving results to /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook05 



Permutations: 100%|██████████| 2/2 [00:00<00:00,  6.73it/s]


Voxelwise FWE p-values shape: (4, 228483)


View and Save the Maps

In [24]:
# Save and visualize results
reg.save_and_visualize_results(verbose=True)  # Change to False to disable visualization

Saving results...


/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/py

All results saved.
